In [1]:
import os, random, sys, matplotlib.pyplot as plt
import torch, torch.nn as nn, numpy
from nltk.tokenize import word_tokenize
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Colab Notebooks/sarcasm.zip" -d "/content"

Archive:  /content/drive/MyDrive/Colab Notebooks/sarcasm.zip
   creating: /content/sarcasm/


In [3]:
def get_data(train_data,test_data):
  train_df = pd.read_csv(train_data,index_col=0)  #training data set
  train_df = train_df.drop(columns=['rephrase','sarcasm','irony','understatement','overstatement', 'rhetorical_question','satire',])
  test_df = pd.read_csv(test_data) # getting the csv file for the test data set  
  return train_df, test_df # getting the train and test data set 

train_df, test_df = get_data('/content/drive/MyDrive/Colab Notebooks/dataset/train.En.csv','/content/drive/MyDrive/Colab Notebooks/dataset/task_A_En_test.csv')


In [4]:
!pip install pytorch-pretrained-bert pytorch-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 72.5 MB/s eta 0:00:00


In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
from sklearn.metrics import classification_report
import io
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def clean_data(df, column):

# cleaning the data to remove any extra characters 
  import re
  from nltk.corpus import stopwords
  import nltk
  import nltk.corpus
  nltk.download('stopwords')
  from nltk.corpus import stopwords
  stop = stopwords.words('english')


  b = list(df[column])
  corpus = []
  for i in range(len(b)):
      review =re.sub(r'http\S+', ' ', str(b[i]))
      review = re.sub("\d*\.\d+","",review)
      review =re.sub(r'@\S+', ' ', review)
      
      TAG_RE = re.compile(r'<[^>]+>')
      review = TAG_RE.sub('', review)

      review = re.sub('    ', ' ', review)
      review = review.lower()
      review = review.split()
      
      review = ' '.join(review)

      

      corpus.append(review)
  df = df.assign(clean = corpus)
  # returns the dataset 
  return df

In [7]:
train_df=clean_data(train_df,"tweet")
train_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,tweet,sarcastic,clean
0,The only thing I got from college is a caffein...,1,the only thing i got from college is a caffein...
1,I love it when professors draw a big question ...,1,i love it when professors draw a big question ...
2,Remember the hundred emails from companies whe...,1,remember the hundred emails from companies whe...
3,Today my pop-pop told me I was not “forced” to...,1,today my pop-pop told me i was not “forced” to...
4,@VolphanCarol @littlewhitty @mysticalmanatee I...,1,"i did too, and i also reported cancun cruz not..."
...,...,...,...
3463,The population spike in Chicago in 9 months is...,0,the population spike in chicago in 9 months is...
3464,You'd think in the second to last English clas...,0,you'd think in the second to last english clas...
3465,I’m finally surfacing after a holiday to Scotl...,0,i’m finally surfacing after a holiday to scotl...
3466,Couldn't be prouder today. Well done to every ...,0,couldn't be prouder today. well done to every ...


In [8]:
def get_sentences_and_labels(df):
  sentences = df.clean.values

  sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
  labels = df.sarcastic.values
  return sentences, labels
sentences, labels = get_sentences_and_labels(train_df)


In [9]:
!pip install pytorch-transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 36.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=75928f1abe22dea1c1886d14af68bf4b46918609d85f7a53804d7689e4ddaba2
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.5 M

In [10]:
def get_tokenized_texts(sentences):
#using the BERT model to get the tokenizers in lower case 
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

  tokenized_texts = [tokenizer.tokenize(sentence) for sentence in sentences]
  return tokenized_texts #tokenize the sentences and print the tokenized texts
tokenized_texts = get_tokenized_texts(sentences)

100%|██████████| 231508/231508 [00:00<00:00, 1259161.43B/s]


In [11]:
def get_input_ids(tokenized):
  # using the BERT model 
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
  max_length = 128
   #pad our input tokens 
  input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized],
                            maxlen=max_length, dtype="long", truncating="post", padding="post")
  # Uses the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized]
  input_ids = pad_sequences(input_ids, maxlen=max_length, dtype="long", truncating="post", padding="post")
  return input_ids
input_ids = get_input_ids(tokenized_texts)

In [12]:
def get_attention_masks(input_ids):
  # Create attention masks
  attention_masks = []

# Creates a mask of 1s for each token followed by 0s for padding
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
  return attention_masks
attention_masks = get_attention_masks(input_ids) # calls the attention masks on the input_ids 


In [13]:
# getting the train validation inputs using input_ids and labels from the BERT model 
def get_train_validation_inputs(input_ids, labels):
  train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                              random_state=2018, test_size=0.1)
  return train_inputs, validation_inputs, train_labels, validation_labels
# getting the train validation masks using the attention masks and input_ids from the BERT model 
def get_train_validation_masks(attention_masks,input_ids):
  train_masks, validation_masks, x, y = train_test_split(attention_masks, input_ids,
                                              random_state=2018, test_size=0.1)
  return train_masks, validation_masks, x, y
# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = get_train_validation_inputs(input_ids, labels)
train_masks, validation_masks, x, y = get_train_validation_masks(attention_masks, input_ids)

In [14]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [37]:
batch_size = 32 # Batch size for training 

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [38]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()
# Loading BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [56]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
# contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [58]:


epochs = 4

for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # trains the data for one epoch 
  for step, batch in enumerate(train_dataloader):
    #adds batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # clear out the gradients if they accumulate by default 
    b_input_ids, b_input_mask, b_labels = batch
    # forward pass 
    optimizer.zero_grad()
    # backward pass 
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # Update parameters and take a step using the computed gradient
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation
# Putting the model in evaluation mode to evaluate loss on the validation set
  model.eval()

  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  predictions , truths = [], []

  for batch in validation_dataloader:
     # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
     # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
     # clear out the gradients if they accumulate by default 
    with torch.no_grad():
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    
    nb_eval_steps += 1
    predictions.append(logits)
    truths.append(label_ids)
    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()    
    flat_truths = [item for sublist in truths for item in sublist]
  

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.43045180853532283


Epoch:  25%|██▌       | 1/4 [01:24<04:14, 84.68s/it]

Train loss: 0.18354405953111697


Epoch:  50%|█████     | 2/4 [02:50<02:50, 85.38s/it]

Train loss: 0.06874214984685639


Epoch:  75%|███████▌  | 3/4 [04:15<01:25, 85.17s/it]

Train loss: 0.03255782272828249


Epoch: 100%|██████████| 4/4 [05:40<00:00, 85.11s/it]


In [59]:
# we are including the test data set to evaluate and see how good our model is comparitively 
test_df = clean_data(test_df, "text")

sentences, labels = get_sentences_and_labels(test_df)

tokenized_texts = get_tokenized_texts(sentences)

input_ids = get_input_ids(tokenized_texts)

attention_masks = get_attention_masks(input_ids)

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32

prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:

# Prediction on test set

model.eval()

predictions , true_labels = [], []

for batch in prediction_dataloader:
    # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
# Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
    # clear out the gradients if they accumulate by default 
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
     # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  predictions.append(logits) # appending the logists to the predictions 
  true_labels.append(label_ids)

In [61]:
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1)
flat_true_labels = [item for sublist in true_labels for item in sublist]
flat_predictions

array([0, 1, 1, ..., 1, 1, 0])

In [62]:
from sklearn.metrics import f1_score, precision_score, recall_score
f1_sarcastic = f1_score(flat_true_labels,flat_predictions, average = "binary", pos_label = 1)
f1_sarcastic # returns the f score

0.28342245989304815

In [24]:

#The model does not generates a .csv file yet, will work for next iteration.

def script(test_file, model_generated_file):
    from sklearn.metrics import f1_score, precision_score, recall_score
    test_df = pd.read_csv(test_data)
    model_df = pd.read_csv(model_generated_file)
    f1_sarcastic = f1_score(test_df['sarcastic'],model_df['sarcastic'], average = "binary", pos_label = 1)
    return f1_sarcastic

# script(test_data,test_data)